pulp 数理最適化

In [1]:
import numpy as np
from itertools import product
from pulp import *

np.random.seed(1)

nw, nf = 3, 4
pr = list(product(range(nw),range(nf)))
供給 = np.random.randint(30, 50, nw)
需要 = np.random.randint(20, 40, nf)
輸送費 = np.random.randint(10, 20, (nw,nf))

m1 = LpProblem('輸送費最適化', sense=LpMinimize)
v1 = {(i, j): LpVariable(f'v{i}_{j}', lowBound=0) for i, j in pr}

m1 += lpSum(輸送費[i][j] * v1[i, j] for i, j in pr)
for i in range(nw):
    m1 += lpSum(v1[i, j] for j in range(nf)) <= 供給[i]
for j in range(nf):
    m1 += lpSum(v1[i, j] for i in range(nw)) >= 需要[j]

status = m1.solve()
print('Status:', LpStatus[status])
{k:value(x) for k,x in v1.items() if value(x) > 0}

Status: Optimal


{(0, 0): 28.0,
 (0, 1): 7.0,
 (1, 2): 31.0,
 (1, 3): 5.0,
 (2, 1): 22.0,
 (2, 3): 20.0}

pandas Version

In [4]:
import numpy as np
import pandas as pd
from itertools import product
from pulp import *


np.random.seed(1)

nw, nf = 3, 4
pr = list(product(range(nw),range(nf)))
supply = np.random.randint(30, 50, nw)
demand = np.random.randint(20, 40, nf)
cost = np.random.randint(10, 20, (nw,nf))

print(pr)

m2 = LpProblem('輸送費最適化pandas', sense=LpMinimize)

df = pd.DataFrame([(i, j) for i, j in pr], columns=['werehouse', 'factory'])
display(df)
df['cost'] = cost.flatten()
display(df)

df['var'] = [LpVariable(f'v{i}', lowBound=0) for i in df.index]
display(df)

m2 += lpDot(df['cost'], df['var'])
m2

for k, v in df.groupby('werehouse'):
    m2 += lpSum(v['var']) <= supply[k]
for k, v in df.groupby('factory'):
    m2 += lpSum(v['var']) >= demand[k]

print(m2)

status = m2.solve()
print('Status:', LpStatus[status])
df['Val'] = df['var'].apply(value)
df['sub_total'] = df['Val'] * df['cost']

print('Without Zero')
display(df[df['Val']>0])

[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3)]


,werehouse,factory
0,0,0
1,0,1
2,0,2
3,0,3
4,1,0
5,1,1
6,1,2
7,1,3
8,2,0
9,2,1


,werehouse,factory,cost
0,0,0,10
1,0,1,10
2,0,2,11
3,0,3,17
4,1,0,16
5,1,1,19
6,1,2,12
7,1,3,14
8,2,0,15
9,2,1,12


,werehouse,factory,cost,var
0,0,0,10,v0
1,0,1,10,v1
2,0,2,11,v2
3,0,3,17,v3
4,1,0,16,v4
5,1,1,19,v5
6,1,2,12,v6
7,1,3,14,v7
8,2,0,15,v8
9,2,1,12,v9


輸送費最適化pandas:
MINIMIZE
10*v0 + 10*v1 + 14*v10 + 12*v11 + 11*v2 + 17*v3 + 16*v4 + 19*v5 + 12*v6 + 14*v7 + 15*v8 + 12*v9 + 0
SUBJECT TO
_C1: v0 + v1 + v2 + v3 <= 35

_C2: v4 + v5 + v6 + v7 <= 41

_C3: v10 + v11 + v8 + v9 <= 42

_C4: v0 + v4 + v8 >= 28

_C5: v1 + v5 + v9 >= 29

_C6: v10 + v2 + v6 >= 31

_C7: v11 + v3 + v7 >= 25

VARIABLES
v0 Continuous
v1 Continuous
v10 Continuous
v11 Continuous
v2 Continuous
v3 Continuous
v4 Continuous
v5 Continuous
v6 Continuous
v7 Continuous
v8 Continuous
v9 Continuous

Status: Optimal
Without Zero


,werehouse,factory,cost,var,Val,sub_total
0,0,0,10,v0,28.0,280.0
1,0,1,10,v1,7.0,70.0
6,1,2,12,v6,31.0,372.0
7,1,3,14,v7,5.0,70.0
9,2,1,12,v9,22.0,264.0
11,2,3,12,v11,20.0,240.0
